In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('train.csv')

In [2]:
#Training data management
data= np.array(data)

#Train test split 80:20
test_datas = data[int(len(data)*0.8):]
train_datas = data[:int(len(data)*0.8)]

#Separating pixel data and label data
train_labels = train_datas[:,0] #label col
train_datas = (train_datas[:,1:] - np.min(train_datas[:,1:]))/(np.max(train_datas[:,1:])-np.min(train_datas[:,1:])) # pixel data, scaled to 0-1

test_labels = test_datas[:,0] #label col
test_datas = (test_datas[:,1:] - np.min(test_datas[:,1:]))/(np.max(test_datas[:,1:])-np.min(test_datas[:,1:])) # pixel data, scaled to 0-1

In [3]:
def sigmoid(x): #sigmoid func to squish all inputs into range 0 to 1
    return 1 / (1 + np.exp(-x))

In [4]:
#Initialization

size=[16, 10]
train_data = train_datas[:10] 
train_label = train_labels
# --------------------------------------

weights = [] #list to store all the weights for every layer
biases = [] #list to store all the biases for every layer

#Randomly initialize weights and biases to append to list
'''
weights.append(np.random.uniform(-0.1,0.1,size=(size[0],len(train_data[0])))) #First layer
biases.append(np.random.uniform(-0.1,0.1,size[0])) 
for i in range(len(size)-1): 
    weights.append(np.random.uniform(-0.1,0.1,size=(size[i+1],size[i]))) #following layers
    biases.append(np.random.uniform(-0.1,0.1,size[i+1])) 
'''

#Try using Xavier/Glorot initialization
for i in range(len(size)): #Initialize weights for each layer
    if i == 0:
        weights.append(np.random.randn(size[0], len(train_data[0])) * np.sqrt(1/len(train_data[0])))
    else:
        weights.append(np.random.randn(size[i], size[i-1]) * np.sqrt(1/size[i-1]))

for i in range(len(size)):  #Initialize biases for each layer
    if i == 0:
        biases.append(np.zeros(size[0])) #First layer biases
    else:
        biases.append(np.zeros(size[i]))  



In [5]:
#Temporarily training on 10 data example for trouble shooting
learning_rate = 0.1
for w in range(1):
    train_data = train_datas[w*10:(w+1)*10] 
    for o in range(10):
        #global cost,z,a,one_hot
        #global Zs,As
        cost = 0

        # Create temporary storage for averaging weights and biases
        temp_weights = [] #list to store all the weights for every layer
        temp_biases = [] #list to store all the biases for every layer

        temp_weights.append(np.zeros(shape=(size[0],len(train_data[0])))) #First layer
        temp_biases.append(np.zeros(size[0])) 
        for i in range(len(size)-1): 
            temp_weights.append(np.zeros(shape=(size[i+1],size[i]))) #following layers
            temp_biases.append(np.zeros(size[i+1])) 


        for i in range(len(train_data)): #Iterate through every train_data
            #Forward propagation
            Zs = []
            As = [train_data[i]] #TAKE NOTE that As and Zs will be different because we put in initial input as first item for QOL during backprop
            z = weights[0] @ train_data[i] + biases[0] #First layer
            a = sigmoid(z)
            Zs.append(z) #Storing data for backward propagation
            As.append(a)
            
            for j in range(len(size)-1): 
                z = weights[j+1] @ a + biases[j+1] #Following layers
                a = sigmoid(z)
                Zs.append(z) #Storing data for backward propagation
                As.append(a)

            #Calculating cost
            
            one_hot = np.zeros(10)
            one_hot[train_label[i]]=1
            
            cost = cost + np.sum((a - one_hot)**2) #Just to keep track of model fit

            #final/output layer Backpropagation
            dC_da = 2*(a - one_hot) 
            #print("Last layer dC_da=",dC_da,"\n")
            dadz = (np.exp(-z) / (1 + np.exp(-z))**2)
            
            for x in range (len(weights[-1][0])): #iterating through weights column by column
                # updating weights              
                dzdw = As[-2][x] #This one input, affects a whole column of weights
                dC_dw = dC_da * dadz * dzdw 

                
                (temp_weights[-1])[:,x] += -dC_dw*learning_rate/len(train_data) #keeping track of updates to the weights
                
                
            #updating Biases
            dzdb = 1
            dC_db = dC_da * dadz * dzdb
            temp_biases[-1] += -dC_db*(learning_rate)/len(train_data) #keeping track of updates to the biases

            #print("Updates to biases=", temp_biases[-1] ) #DEBUGGING

            global dCda_0 
            #Previous layer Backpropagation
            dCda_0 = np.array([])
            for x in range (len(weights[-1][0])): #iterating through inputs, a, summing weights column by column, 
                dzda_0 = weights[-1][:,x] #A whole column of weights affect how ONE prev layer input affects the next layer 
                dC_da_0 = np.sum(dC_da*dadz*dzda_0)/len(weights[-1]) #Keep track of how previous layer output affect next layer for chain rule later
                dCda_0 = np.append(dCda_0,dC_da_0)
            #print("second from last layer dCda=\n",dCda_0)

            #Previous layer weights
            for k in range(len(size)-1): #iterating through layers, starting from the second last
                z = Zs[-k-2]
                dadz = (np.exp(-z) / (1 + np.exp(-z))**2)
                
                #Updating previous layer weights
                for l in range (len(weights[-2-k][0])): #iterating through weights column by column (-2-k because we start from second from last)
                    
                    dzdw = As[-3-k][l] #This one input, affects a whole column of weights
                    dC_dw = dCda_0 * dadz * dzdw
                    
                    (temp_weights[-2-k])[:,l] += -dC_dw*(learning_rate)/len(train_data) #keeping track of updates to the weights


                #updating Biases
                dzdb = 1
                dC_db = dCda_0 * dadz * dzdb
                temp_biases[-2-k] += -dC_db*(learning_rate)/len(train_data) #keeping track of updates to the biases

                #Keep track of how this layer output affect next layer for chain rule later
                temp_dCda_0 = np.array([])
                for x in range (len(weights[-2-k][0])): #iterating through inputs, a, summing weights column by column
                    dzda_0 = weights[-2-k][:,x] #A whole column of weights affect how ONE prev layer input affects the next layer 
                    dC_da_0 = np.sum(dCda_0*dadz*dzda_0)/len(weights[-2-k]) 
                    temp_dCda_0 = np.append(temp_dCda_0,dC_da_0)
                
                dCda_0 = temp_dCda_0 #MUtable / unmutable object? Is this going to be problem?

        #Updating biases and weights

        for i in range(len(size)):
            weights[i] += temp_weights[i]
            biases[i] += temp_biases[i]

        # Analysis of changes to weights 
        print("weights, iteration",o)
        print(temp_weights[0][0][132:136])

        print("\n", weights[0][0][132:136])

        print("\n",temp_weights[1][0])

        print("\n", weights[1][0])

        # Analysis of changes to biases 
        print("biases, iteration",o)
        print("\n",temp_biases[0])

        print("\n", biases[0])

        print("\n", temp_biases[1])

        print("\n", biases[1])

        
                



weights, iteration 0
[1.30862550e-05 1.77499735e-05 6.54312749e-06 0.00000000e+00]

 [ 0.0467435  -0.03802533  0.02486913  0.06770515]

 [-0.0071969  -0.0054507  -0.0046609  -0.00588062 -0.00549288 -0.00556801
 -0.00643707 -0.00658048 -0.00552199 -0.00539419 -0.00576723 -0.00547148
 -0.00584958 -0.0045769  -0.00574469 -0.00492983]

 [-0.47730139  0.17890558  0.48633069 -0.19608    -0.12501675 -0.06667329
  0.0677023  -0.30814525 -0.03055555 -0.04629964 -0.37025132  0.20639895
  0.56576327  0.25252925  0.07828255 -0.0693977 ]
biases, iteration 0

 [ 5.07614219e-05 -4.44027895e-04 -2.89423996e-04  1.65995244e-04
  1.93724464e-04 -1.44972201e-04 -3.29163432e-04 -1.32856664e-04
 -2.77981933e-04  3.93367722e-05  1.64248949e-04 -6.34226906e-05
  4.85612341e-04 -4.00647835e-04 -2.24374585e-04  1.17076355e-04]

 [ 5.07614219e-05 -4.44027895e-04 -2.89423996e-04  1.65995244e-04
  1.93724464e-04 -1.44972201e-04 -3.29163432e-04 -1.32856664e-04
 -2.77981933e-04  3.93367722e-05  1.64248949e-04 -6.34

In [6]:
cost

14.08779937291723

In [7]:
#Forward propagation, testing training fit
m=0
z = weights[0] @ train_datas[m] + biases[0] #First layer
a = sigmoid(z)
print("\nFirst layer, \nz=",z,"\na=",a )

for j in range(len(size)-1): 
    z = weights[j+1] @ a + biases[j+1] #Following layers
    a = sigmoid(z)
    print("\n",j+1,"th layer, \nz=",z,"\na=",a )

print("\nevaluation=",a,"max= ",np.argmax(a)," label= ",train_labels[m])


First layer, 
z= [ 0.59352671 -0.07510365  0.00176616  0.32869294 -0.03007599 -0.27031898
  0.13036989  0.27099842  0.36335949 -0.28867033  0.2827137   0.1660659
 -0.03329005 -0.36498986  0.09094213  0.21033653] 
a= [0.64417393 0.48123291 0.50044154 0.58144131 0.49248157 0.43282879
 0.53254639 0.567338   0.58985343 0.42832942 0.5702114  0.54142133
 0.49167826 0.4097522  0.52271987 0.55239112]

 1 th layer, 
z= [-0.46599064 -0.34925698 -0.9116827  -0.87353634 -0.06009452 -1.09594521
 -1.19446317 -1.1359905  -0.52587584 -1.46083889] 
a= [0.38556564 0.41356261 0.28665563 0.294519   0.48498089 0.25050041
 0.23246165 0.24305727 0.37147929 0.18833905]

evaluation= [0.38556564 0.41356261 0.28665563 0.294519   0.48498089 0.25050041
 0.23246165 0.24305727 0.37147929 0.18833905] max=  4  label=  1


In [8]:
#Forward propagation, testing training fit
m=4
z = weights[0] @ train_datas[m] + biases[0] #First layer
a = sigmoid(z)
print("\nFirst layer, \nz=",z,"\na=",a )

for j in range(len(size)-1): 
    z = weights[j+1] @ a + biases[j+1] #Following layers
    a = sigmoid(z)
    print("\n",j+1,"th layer, \nz=",z,"\na=",a )

print("\nevaluation=",a,"max= ",np.argmax(a)," label= ",train_labels[m])


First layer, 
z= [ 0.57247577 -0.88956797  0.91471418 -0.54673857  0.05350222 -0.61355624
  0.14600472 -0.2834494   0.56115173  0.02980281 -0.53781147  0.28410036
  0.50838543  0.1264589  -0.00230475  0.93432693] 
a= [0.63933425 0.29119899 0.71396386 0.36662142 0.51337237 0.3512484
 0.53643648 0.42960831 0.63671899 0.50745015 0.36869684 0.57055119
 0.62442791 0.53157266 0.49942381 0.7179523 ]

 1 th layer, 
z= [-0.12874073 -0.49599182 -0.94756282 -0.79446769 -0.0513548  -1.09239514
 -1.31056131 -1.14918699 -0.51008878 -1.6286352 ] 
a= [0.4678592  0.37848307 0.27937522 0.31121018 0.48716412 0.25116753
 0.21239293 0.24063761 0.37517271 0.16401741]

evaluation= [0.4678592  0.37848307 0.27937522 0.31121018 0.48716412 0.25116753
 0.21239293 0.24063761 0.37517271 0.16401741] max=  4  label=  0


In [ ]:
#Check accuracy on training set
correct = 0
k = 10
for i in range(k):
    z = weights[0] @ train_datas[i] + biases[0] #First layer
    a = sigmoid(z)

    for j in range(len(size)-1): 
        z = weights[j+1] @ a + biases[j+1] #Following layers
        a = sigmoid(z)

    if train_labels[i] == np.argmax(a): #np.argmax(a)
        correct += 1
        
print(correct/k)

0.0
